# Lecture 2 Demo: Quantum Gates, Circuits, and Entanglement

Computational companion notebook for Lecture 2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

import pennylane as qml

print(f"Qiskit version: {__import__('qiskit').__version__}")
print(f"PennyLane version: {qml.__version__}")

## Demo 0. Circuit visualization

This example highlights wire order, control and target qubits, and circuit depth in a multi-qubit circuit.

The same circuit can be visualized with `qc.draw(...)` and reproduced in IBM Quantum Composer.

In [ ]:
qc_composer = QuantumCircuit(3)
qc_composer.h(0)
qc_composer.cx(0, 1)
qc_composer.cx(1, 2)
qc_composer.barrier()
qc_composer.ry(np.pi / 3, 2)
qc_composer.measure_all()

print(qc_composer.draw("text"))

# Matplotlib drawer gives a slide-friendly circuit picture.
qc_composer.draw("mpl")

## Demo 1. Single-qubit gates on basis states

Every qubit starts in $\lvert 0\rangle$ by default unless explicitly prepared otherwise.

In [ ]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

print("X|0> =", X @ ket_0, "   X|1> =", X @ ket_1)
print("Y|0> =", Y @ ket_0, "   Y|1> =", Y @ ket_1)
print("Z|0> =", Z @ ket_0, "   Z|1> =", Z @ ket_1)
print("H|0> =", np.round(H @ ket_0, 4))
print("H|1> =", np.round(H @ ket_1, 4))

## Demo 2. Bell state construction step by step

Deterministic state evolution:
$\lvert 00\rangle \xrightarrow{H\otimes I} (\lvert 00\rangle + \lvert 10\rangle)/\sqrt{2} \xrightarrow{\mathrm{CNOT}} (\lvert 00\rangle + \lvert 11\rangle)/\sqrt{2}$.

In [ ]:
basis_labels = ["00", "01", "10", "11"]

state = Statevector.from_label("00")
snapshots = [("initial |00>", state.probabilities().copy())]

qc_h = QuantumCircuit(2)
qc_h.h(0)
state = state.evolve(qc_h)
snapshots.append(("after H(q0)", state.probabilities().copy()))

qc_cx = QuantumCircuit(2)
qc_cx.cx(0, 1)
state = state.evolve(qc_cx)
snapshots.append(("after CX(0,1)", state.probabilities().copy()))

for label, probs in snapshots:
    p = dict(zip(basis_labels, np.round(probs, 4)))
    print(f"{label:18s}  P = {p}")

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
for idx, (title, probs) in enumerate(snapshots):
    ax = axes[idx]
    vals = [probs[i] for i in range(4)]
    colors = ["steelblue" if v > 0 else "lightgray" for v in vals]
    ax.bar(basis_labels, vals, color=colors)
    ax.set_title(title, fontsize=12)
    ax.set_ylim(0, 1.05)
    ax.set_ylabel("probability")

plt.suptitle("State evolution to Bell state", fontsize=13)
plt.tight_layout()
plt.show()

## Demo 3. All four Bell states (Qiskit)

The statevector evolution is deterministic; randomness appears at measurement.

In [ ]:
def make_bell_circuits() -> dict[str, QuantumCircuit]:
    """Return circuits preparing the four Bell states."""
    specs = {
        "Phi+": [("h", 0), ("cx", (0, 1))],
        "Phi-": [("h", 0), ("z", 0), ("cx", (0, 1))],
        "Psi+": [("h", 0), ("cx", (0, 1)), ("x", 1)],
        "Psi-": [("h", 0), ("z", 0), ("cx", (0, 1)), ("x", 1)],
    }
    circuits = {}
    for name, gates in specs.items():
        qc = QuantumCircuit(2, name=name)
        for gate, args in gates:
            if isinstance(args, tuple):
                getattr(qc, gate)(*args)
            else:
                getattr(qc, gate)(args)
        circuits[name] = qc
    return circuits


sampler = StatevectorSampler()
bell_circuits = make_bell_circuits()

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for idx, (name, qc) in enumerate(bell_circuits.items()):
    qc_meas = qc.copy()
    qc_meas.measure_all()
    counts = sampler.run([qc_meas], shots=1024).result()[0].data.meas.get_counts()

    ax = axes.flat[idx]
    labels = sorted(counts.keys())
    vals = [counts[l] for l in labels]
    ax.bar(labels, vals, color=["steelblue", "coral", "seagreen", "orange"][:len(labels)])
    ax.set_title(f"|{name}>", fontsize=13)
    ax.set_ylim(0, 1100)
    ax.set_ylabel("counts")

plt.suptitle("Bell states (1024 shots)", fontsize=14)
plt.tight_layout()
plt.show()

## Demo 4. GHZ state and multi-qubit visualization

A 3-qubit GHZ state extends Bell correlations to three parties.

In [ ]:
qc_ghz = QuantumCircuit(3)
qc_ghz.h(0)
qc_ghz.cx(0, 1)
qc_ghz.cx(0, 2)

print(qc_ghz.draw("text"))

sv_ghz = Statevector.from_instruction(qc_ghz)
print("\nNon-zero amplitudes:")
for idx, amp in enumerate(sv_ghz.data):
    if abs(amp) > 1e-10:
        print(f"  |{idx:03b}>: {amp}")

qc_ghz_meas = qc_ghz.copy()
qc_ghz_meas.measure_all()
counts = sampler.run([qc_ghz_meas], shots=2048).result()[0].data.meas.get_counts()
print("\nCounts:", counts)
plot_histogram(counts)

## Demo 5. Why deterministic gates still give probabilistic outcomes

Gate application is deterministic linear algebra on amplitudes.
Measurement returns one sampled outcome, so individual shots are random.
The estimated probabilities converge with more shots.

In [ ]:
dev = qml.device("default.qubit", wires=1)


@qml.qnode(dev)
def ry_expval(theta):
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))


theta = 1.1
exact_expval = float(ry_expval(theta))
exact_p0 = (1 + exact_expval) / 2
exact_p1 = 1 - exact_p0
print(f"Exact <Z> = {exact_expval:.6f}")
print(f"Exact probabilities: P(0)={exact_p0:.6f}, P(1)={exact_p1:.6f}")

qc = QuantumCircuit(1)
qc.ry(theta, 0)
qc.measure_all()

shots_list = [64, 256, 1024, 4096]
p0_estimates = []
for shots in shots_list:
    counts = sampler.run([qc], shots=shots).result()[0].data.meas.get_counts()
    p0_hat = counts.get("0", 0) / shots
    p0_estimates.append(p0_hat)
    print(f"shots={shots:4d}  counts={counts}  p0_hat={p0_hat:.4f}")

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(shots_list, p0_estimates, "o-", lw=2, label="estimate from sampling")
ax.axhline(exact_p0, color="red", ls="--", label="exact P(0)")
ax.set_xscale("log", base=2)
ax.set_xlabel("shots")
ax.set_ylabel("P(0)")
ax.set_title("Sampling converges to deterministic probabilities")
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()